<h3>📌 Train Notebook:</h3> <h4><a href='https://www.kaggle.com/code/debarshichanda/pytorch-feedback-deberta-v3-baseline'>https://www.kaggle.com/code/debarshichanda/pytorch-feedback-deberta-v3-baseline</a></h4>

* cv0.6819
* lb:652
* here code:https://www.kaggle.com/code/quincyqiang/feedback-meanpoolingv2-inference
* thanks to:Debarshi Chanda
* hits：Based on the baseline, trying the way of text splicing:add `discourse_type` and training on 5folds

```
text = discourse_type+self.tokenizer.sep_token+discourse +self.tokenizer.sep_token + " " + essay
```

In [ ]:
import os
import gc
import cv2
import copy
import time
import random
import joblib

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# For Transformer Models
from transformers import AutoTokenizer, AutoModel, AutoConfig

# Utils
from tqdm import tqdm

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
MODEL_DIR='../input/debertav3base-mpv-cv5'
MODEL_PATHS = [
    f'{MODEL_DIR}/Loss-Fold-0.bin',
    f'{MODEL_DIR}/Loss-Fold-1.bin',
    f'{MODEL_DIR}/Loss-Fold-2.bin',
    f'{MODEL_DIR}/Loss-Fold-3.bin',
    f'{MODEL_DIR}/Loss-Fold-4.bin',
]

In [ ]:
TRAIN_DIR = "../input/feedback-prize-effectiveness/train"
TEST_DIR = "../input/feedback-prize-effectiveness/test"

In [ ]:
CONFIG = dict(
    seed = 42,
    model_name = '../input/debertav3base',
    test_batch_size = 16,
    max_length = 512,
    num_classes = 3,
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
)

CONFIG["tokenizer"] = AutoTokenizer.from_pretrained(CONFIG['model_name'])

In [ ]:
def get_essay(essay_id):
    essay_path = os.path.join(TEST_DIR, f"{essay_id}.txt")
    essay_text = open(essay_path, 'r').read()
    return essay_text

In [ ]:
df = pd.read_csv("../input/feedback-prize-effectiveness/test.csv")
df['essay_text'] = df['essay_id'].apply(get_essay)
df.head()

In [ ]:
with open(f"../input/{MODEL_DIR}/le.pkl", "rb") as fp:
    encoder = joblib.load(fp)
    
encoder.classes_

In [ ]:
class FeedBackDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.discourse_type = df['discourse_type'].values
        self.discourse = df['discourse_text'].values
        self.essay = df['essay_text'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        discourse_type = self.discourse_type[index]
        discourse = self.discourse[index]
        
        essay = self.essay[index]
        text = discourse_type+self.tokenizer.sep_token+discourse +self.tokenizer.sep_token + " " + essay
        inputs = self.tokenizer.encode_plus(
                        text,
                        truncation=True,
                        add_special_tokens=True,
                        max_length=self.max_len,
                        padding='max_length'
                    )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long)
        }

In [ ]:
test_dataset = FeedBackDataset(df, CONFIG['tokenizer'], max_length=CONFIG['max_length'])
test_loader = DataLoader(test_dataset, batch_size=CONFIG['test_batch_size'],
                         num_workers=2, shuffle=False, pin_memory=True)

In [ ]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

In [ ]:
class FeedBackModel(nn.Module):
    def __init__(self, model_name):
        super(FeedBackModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.config = AutoConfig.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.2)
        self.pooler = MeanPooling()
        self.fc = nn.Linear(self.config.hidden_size, CONFIG['num_classes'])
        
    def forward(self, ids, mask):        
        out = self.model(input_ids=ids,attention_mask=mask,
                         output_hidden_states=False)
        out = self.pooler(out.last_hidden_state, mask)
        out = self.drop(out)
        outputs = self.fc(out)
        return outputs

In [ ]:
@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        
        outputs = model(ids, mask)
        outputs = F.softmax(outputs, dim=1)
        PREDS.append(outputs.cpu().detach().numpy()) 
    
    PREDS = np.concatenate(PREDS)
    gc.collect()
    
    return PREDS

In [ ]:
def inference(model_paths, dataloader, device):
    final_preds = []
    for i, path in enumerate(model_paths):
        model = FeedBackModel(CONFIG['model_name'])
        model.to(CONFIG['device'])
        model.load_state_dict(torch.load(path))
        
        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
    
    final_preds = np.array(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    return final_preds

In [ ]:
preds = inference(MODEL_PATHS, test_loader, CONFIG['device'])

In [ ]:
preds

In [ ]:
sample = pd.read_csv("../input/feedback-prize-effectiveness/sample_submission.csv")
sample.head()

In [ ]:
sample['Adequate'] = preds[:, 0]
sample['Effective'] = preds[:, 1]
sample['Ineffective'] = preds[:, 2]

sample.head()

In [ ]:
sample.to_csv('submission.csv', index=False)